<a href="https://colab.research.google.com/github/adammoss/bnn_hmc/blob/main/results/CMD_MCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
samples_iter = 200
dropout = 0.1

In [3]:
!pip install astro-datasets --upgrade
!pip install tensorflow_datasets --upgrade
import astro_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
ds_train, info_train = tfds.load(name='cmd', split='train[:90%]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'IllustrisTNG', 'field': 'Mtot', 'parameters': ['omegam']})
ds_valid, info_valid = tfds.load(name='cmd', split='train[90%:95%]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'IllustrisTNG', 'field': 'Mtot','parameters': ['omegam']})
ds_test, info_test = tfds.load(name='cmd', split='train[95%:]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'IllustrisTNG', 'field': 'Mtot','parameters': ['omegam']})

In [5]:
minimum = np.array([0.1, 0.6])
maximum = np.array([0.5, 1.0])  

def normalize(image, label):  
  image = tf.math.log(image)
  label = (label - minimum[0])/(maximum[0] - minimum[0])
  return image, label

ds_train = ds_train.map(normalize)
ds_valid = ds_valid.map(normalize)
ds_test = ds_test.map(normalize)

def rotate_map(image, label):
    return tf.image.rot90(image, np.random.choice([-1,0,1,2])), label

#ds_train = ds_train.map(rotate_map)

In [6]:
class MonteCarloDropout(tf.keras.layers.Dropout):
     def call(self, inputs):
         return super().call(inputs, training=True)

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=5, strides=1, padding = 'same', input_shape=(256, 256, 1)),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2,2)),
    MonteCarloDropout(dropout),
    tf.keras.layers.Conv2D(16, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2,2)),
    MonteCarloDropout(dropout),
    tf.keras.layers.Conv2D(120, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2,2)),
    MonteCarloDropout(dropout),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84),
    tf.keras.layers.ReLU(),
    MonteCarloDropout(dropout),
    tf.keras.layers.Dense(1, activation='linear'),
    ])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 6)       156       
                                                                 
 re_lu (ReLU)                (None, 256, 256, 6)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 6)      0         
 )                                                               
                                                                 
 monte_carlo_dropout (MonteC  (None, 127, 127, 6)      0         
 arloDropout)                                                    
                                                                 
 conv2d_1 (Conv2D)           (None, 127, 127, 16)      2416      
                                                                 
 re_lu_1 (ReLU)              (None, 127, 127, 16)      0

In [9]:
opt = tf.keras.optimizers.Adam(learning_rate=5e-5)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=3)

In [10]:
model.compile(optimizer=opt,
              loss='MeanSquaredError',
              metrics='mae')

In [11]:
checkpoint_path = "cmd/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=False,
                                                  monitor='val_loss',
                                                  mode='min',
                                                  verbose=1,
                                                  save_best_only=True)

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

In [12]:
model.fit(ds_train, epochs=100, callbacks=[reduce_lr, cp_callback, es_callback], validation_data=ds_valid)

Epoch 1/100
179/180 [============================>.] - ETA: 0s - loss: 0.1482 - mae: 0.2733
Epoch 1: val_loss improved from inf to 0.07580, saving model to cmd/cp.ckpt
180/180 [==============================] - 21s 93ms/step - loss: 0.1476 - mae: 0.2727 - val_loss: 0.0758 - val_mae: 0.2139 - lr: 5.0000e-05
Epoch 2/100
179/180 [============================>.] - ETA: 0s - loss: 0.0623 - mae: 0.1838
Epoch 2: val_loss improved from 0.07580 to 0.05128, saving model to cmd/cp.ckpt
180/180 [==============================] - 16s 85ms/step - loss: 0.0623 - mae: 0.1837 - val_loss: 0.0513 - val_mae: 0.1629 - lr: 5.0000e-05
Epoch 3/100
179/180 [============================>.] - ETA: 0s - loss: 0.0520 - mae: 0.1690
Epoch 3: val_loss improved from 0.05128 to 0.04710, saving model to cmd/cp.ckpt
180/180 [==============================] - 27s 150ms/step - loss: 0.0519 - mae: 0.1688 - val_loss: 0.0471 - val_mae: 0.1690 - lr: 5.0000e-05
Epoch 4/100
179/180 [============================>.] - ETA: 0s - lo

In [52]:
new_model = tf.keras.models.load_model('cmd/cp.ckpt')

In [53]:
def unnormalize(image, label):  
  image = tf.math.exp(image)
  label = (label*(maximum[0] - minimum[0])) + minimum[0]
  return image, label

In [75]:
mapes = []
for _ in range(samples_iter):
  y = []
  yhat = []
  for (x_batch, y_batch) in ds_test:
      yhat.append(new_model.predict(x_batch))
      y.append(y_batch)
  y = np.concatenate(y)
  yhat = np.concatenate(yhat)
  # Unnormalise 
  y = (y*(maximum[0] - minimum[0])) + minimum[0]
  yhat = (yhat*(maximum[0] - minimum[0])) + minimum[0]
  print('MAPE', 100*np.mean(np.abs((yhat-y)/y)))
  mapes.append(100*np.mean(np.abs((yhat-y)/y)))

MAPE 15.833335897844858
MAPE 16.082553020360713
MAPE 15.882324393002396
MAPE 16.01743622334023
MAPE 16.27323801094431
MAPE 16.068470238061792
MAPE 15.957168720627857
MAPE 16.200494325090354
MAPE 16.070834450843574
MAPE 16.431896296386935
MAPE 15.830154921466447
MAPE 15.607417900724926
MAPE 15.95901869197561
MAPE 16.342722463431013
MAPE 16.070441389094697
MAPE 16.123381036722286
MAPE 15.867859273764633
MAPE 16.01821902059264
MAPE 16.036182502031863
MAPE 16.44965103665906
MAPE 16.299988727920407
MAPE 16.087026182617898
MAPE 16.052537933583526
MAPE 15.689966250434997
MAPE 16.269275374646575
MAPE 16.74542898916298
MAPE 16.337038739101413
MAPE 16.05657701162506
MAPE 16.086640069607316
MAPE 15.95112628945802
MAPE 16.41958343803041
MAPE 15.841158755779116
MAPE 15.92123792652238
MAPE 16.63937170468592
MAPE 15.838127744997848
MAPE 16.260708111441186
MAPE 16.28449632567038
MAPE 16.210017715513974
MAPE 16.086721877660594
MAPE 16.224253832980327
MAPE 15.837158768903754
MAPE 15.873858502091851
MAPE

In [76]:
print('MAPE', np.mean(mapes), np.std(mapes))

MAPE 16.099338392042778 0.26543405750519866


In [77]:
ds_test_S, info_test_S = tfds.load(name='cmd', split='train[95%:]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'SIMBA', 'field': 'Mtot','parameters': ['omegam']})
ds_test_S = ds_test_S.map(normalize)

In [79]:
mapes = []
for _ in range(samples_iter):
  y = []
  yhat = []
  for (x_batch, y_batch) in ds_test_S:
      yhat.append(new_model.predict(x_batch))
      y.append(y_batch)
  y = np.concatenate(y)
  yhat = np.concatenate(yhat)
  # Unnormalise 
  y = (y*(maximum[0] - minimum[0])) + minimum[0]
  yhat = (yhat*(maximum[0] - minimum[0])) + minimum[0]
  print('MAPE', 100*np.mean(np.abs((yhat-y)/y)))
  mapes.append(100*np.mean(np.abs((yhat-y)/y)))

MAPE 18.890772839892193
MAPE 18.342394665049262
MAPE 18.65599967610351
MAPE 18.817906867149457
MAPE 18.56133920625483
MAPE 18.659977989451985
MAPE 18.941020481423593
MAPE 18.860893211791428
MAPE 18.28527556343545
MAPE 18.86719030302019
MAPE 18.877840405154938
MAPE 18.409637057089533
MAPE 18.494238855759104
MAPE 19.57235110584787
MAPE 18.862770100499596
MAPE 19.176564998733785
MAPE 18.796012163704518
MAPE 18.93237327606552
MAPE 19.03046158955676
MAPE 18.698687232200058
MAPE 18.302099063011777
MAPE 18.819971239166883
MAPE 18.726820455517707
MAPE 18.38105188291855
MAPE 18.60428350069788
MAPE 18.90648272370232
MAPE 19.19160001617695
MAPE 18.863965871546355
MAPE 18.75688390497967
MAPE 18.427218156966
MAPE 18.33470346569073
MAPE 18.648598685434926
MAPE 18.522821676180605
MAPE 18.701555341977
MAPE 18.559032300545134
MAPE 18.66857053637625
MAPE 18.625885369400386
MAPE 18.686018124430294
MAPE 18.97711120674829
MAPE 18.686915476629366
MAPE 18.654456504789806
MAPE 18.46428301482899
MAPE 19.279646

In [80]:
print('MAPE', np.mean(mapes), np.std(mapes))

MAPE 18.79255751889208 0.2402143000483462
